In [2]:
!pip install helper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for helper: filename=helper-2.5.0-py2.py3-none-any.whl size=19188 sha256=331ca1605983fcf5a356e6efd799bdd65766edcd1d1ab53f453039c390e82f41
  Stored in directory: /root/.cache/pip/wheels/85/68/52/33a3eed6a95667d7b9a38afeee13928e3f95912753f1120633
Successfully built helper


In [3]:
import collections

import helper
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from tensorflow.keras.layers import Embedding

In [4]:
import pandas as pd

In [5]:
english = pd.read_csv('https://raw.githubusercontent.com/Kulbear/deep-learning-nano-foundation/master/DLND-language-translation/data/small_vocab_en',sep=';',names=['data'])

In [5]:
english

,data
0,"new jersey is sometimes quiet during autumn , ..."
1,the united states is usually chilly during jul...
2,"california is usually quiet during march , and..."
3,the united states is sometimes mild during jun...
4,"your least liked fruit is the grape , but my l..."
...,...
137855,"france is never busy during march , and it is ..."
137856,"india is sometimes beautiful during spring , a..."
137857,"india is never wet during summer , but it is s..."
137858,"france is never chilly during january , but it..."


In [6]:
french = pd.read_csv('https://raw.githubusercontent.com/Kulbear/deep-learning-nano-foundation/master/DLND-language-translation/data/small_vocab_fr',sep=';',names=['data'])

In [7]:
french

,data
0,new jersey est parfois calme pendant l' automn...
1,les états-unis est généralement froid en juill...
2,"california est généralement calme en mars , et..."
3,"les états-unis est parfois légère en juin , et..."
4,"votre moins aimé fruit est le raisin , mais mo..."
...,...
137855,"la france est jamais occupée en mars , et il e..."
137856,"l' inde est parfois belle au printemps , et il..."
137857,"l' inde est jamais mouillé pendant l' été , ma..."
137858,"la france est jamais froid en janvier , mais i..."


In [7]:
for i in range(5):
    print('English sample {}:  {}'.format(i + 1, english['data'][i]))
    print('French sample {}:  {}\n'.format(i + 1, french['data'][i]))   

English sample 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
French sample 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .

English sample 2:  the united states is usually chilly during july , and it is usually freezing in november .
French sample 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .

English sample 3:  california is usually quiet during march , and it is usually hot in june .
French sample 3:  california est généralement calme en mars , et il est généralement chaud en juin .

English sample 4:  the united states is sometimes mild during june , and it is cold in september .
French sample 4:  les états-unis est parfois légère en juin , et il fait froid en septembre .

English sample 5:  your least liked fruit is the grape , but my least liked is the apple .
French sample 5:  votre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .



In [8]:
def sentence_tokenize(lst_sentences):
  tok = Tokenizer()
  tok.fit_on_texts(lst_sentences)
  return tok.texts_to_sequences(lst_sentences),tok

In [9]:
def sentence_padding(x,length=None):
  return pad_sequences(x, maxlen=length, padding='post')

In [10]:
def preprocess(input,target):
  input_seq,input_tokenizer = sentence_tokenize(input)
  target_seq, target_tokenizer = sentence_tokenize(target)
  input_seq = sentence_padding(input_seq)
  target_seq = sentence_padding(target_seq)

  target_seq = target_seq.reshape(*target_seq.shape, 1)
  return input_seq, target_seq, input_tokenizer, target_tokenizer

In [11]:
preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer = preprocess(english['data'], french['data'])

In [12]:
def inverse_to_text(logits,tokenizer):
  index_to_words = {id: word for word, id in tokenizer.word_index.items()}
  index_to_words[0] = ''
  return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
  #Returns the indices of the maximum values along an axis.

In [14]:
english_tokenizer.sequences_to_texts([preproc_english_sentences[0]])

['new jersey is sometimes quiet during autumn and it is snowy in april']

In [13]:
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [16]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    # Hyperparameters
    learning_rate = 0.005
    
    # TODO: Build the layers
    model = Sequential()
    model.add(GRU(256, input_shape=input_shape[1:], return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

# Reshaping the input to work with a basic RNN

tmp_x = sentence_padding(preproc_english_sentences,max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    english_vocab_size,
    french_vocab_size)

print(simple_rnn_model.summary())

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(inverse_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 21, 256)           198912    
                                                                 
 time_distributed (TimeDistr  (None, 21, 1024)         263168    
 ibuted)                                                         
                                                                 
 dropout (Dropout)           (None, 21, 1024)          0         
                                                                 
 time_distributed_1 (TimeDis  (None, 21, 344)          352600    
 tributed)                                                       
                                                                 
Total params: 814,680
Trainable params: 814,680
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
108/108 [=========================

In [16]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    # Hyperparameters
    learning_rate = 0.002
    
    # TODO: Build the layers
    model = Sequential()
    model.add(LSTM(256, input_shape=input_shape[1:], return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.2))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

# Reshaping the input to work with a basic RNN

tmp_x = sentence_padding(preproc_english_sentences,max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    english_vocab_size,
    french_vocab_size)

print(simple_rnn_model.summary())

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=25, validation_split=0.2)

# Print prediction(s)
print(inverse_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 21, 256)           264192    
                                                                 
 time_distributed_4 (TimeDis  (None, 21, 1024)         263168    
 tributed)                                                       
                                                                 
 dropout_2 (Dropout)         (None, 21, 1024)          0         
                                                                 
 time_distributed_5 (TimeDis  (None, 21, 344)          352600    
 tributed)                                                       
                                                                 
Total params: 879,960
Trainable params: 879,960
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/25
108/108 [=======================

In [21]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    # Hyperparameters
    learning_rate = 0.003
    
    # TODO: Build the layers
    model = Sequential()
    #model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=[input_shape[1:]]))
    model.add(Bidirectional(GRU(128, return_sequences=True)))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model


# TODO: Reshape the input
tmp_x = sentence_padding(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train and Print prediction(s)
embed_rnn_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

embed_rnn_model.summary()

embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(inverse_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10


ValueError: ignored